In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Download PyTorch
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

In [ ]:
# Using the yolov5 algorithm created by ultralytics
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# Installing the dependencies required
!cd yolov5 & pip install -r /content/yolov5/requirements.txt
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
# Importing required libraries
import torch
import PIL
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from os import listdir
from os.path import isfile, join
import pandas
import collections as cl

In [ ]:
# Training the model
!python /content/yolov5/train.py --img 640 --batch 16 --epochs 2 --data /content/yolov5/dataset.yml --weights yolov5x.pt

In [ ]:
# Loading the model to be used
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5/runs/train/exp/weights/best.pt', force_reload=True)

In [ ]:
# Creating a dictionary to store the test images' index and name
f = open("/content/drive/MyDrive/Qualifiers/CV Final Evaluation Dataset/qualifiers_finals_no_annotations.json")
interim_json = json.load(f)
file_names = {}
for i in interim_json["images"]:
  file_names[i["id"]] = i["file_name"]
file_names

In [ ]:
# Generating predictions for the test images using the model created earlier
# and storing the results in a list
ans = []

f = open("/content/drive/MyDrive/Qualifiers/CV Final Evaluation Dataset/qualifiers_finals_no_annotations.json")
interim_json = json.load(f)

for ind in file_names:
  file_str = "/content/drive/MyDrive/Qualifiers/CV Final Evaluation Dataset/Images/" + file_names[ind]
  img = PIL.Image.open(file_str)
  results = model(img)
  results.print()
  results.render()
  res = results.pandas().xyxy[0]

  if len(res.values) == 0:
    d = cl.OrderedDict()
    d["image_id"] = ind
    d["bbox"] = [0.5, 0.5, 0.5, 0.5]
    d["category_id"] = 1
    d["score"] = 0.123456
    ans.append(d)

  for i in range(len(res.values)):
    d = cl.OrderedDict()
    d["image_id"] = ind
    d["bbox"] = []
    d["category_id"] = res.values[i][5]
    d["score"] = round(res.values[i][4], 6)

    d["bbox"].append(round(res.values[i][0], 1))
    d["bbox"].append(round(res.values[i][1], 1))
    width = round((res.values[i][2] - res.values[i][0]), 1)
    height = round((res.values[i][3] - res.values[i][1]), 1)
    d["bbox"].append(width)
    d["bbox"].append(height)
    ans.append(d)

In [ ]:
# Converting the list containing the results into json and output it as a json file
ans = json.dumps(ans, indent = 4)
final_ans = json.loads(ans)
with open("cv_pred.json", "w") as write_file:
    json.dump(final_ans, write_file)